In [ ]:
# Izzet Emre Kucukkaya
# CMPE597 HW1

In [2]:
# Read dataset

In [16]:
from tensorflow.keras import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable

In [5]:
# MNIST dataset
dataset = datasets.mnist.load_data(path="mnist.npz")
X_train = dataset[0][0]
Y_train = dataset[0][1]
X_test = dataset[1][0]
Y_test = dataset[1][1]

X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)


root = './data'
if not os.path.exists(root):
    os.mkdir(root)
    
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)

batch_size = 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

0.4%

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



102.8%

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw




5.8%

100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



112.7%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



### Using Own Implementation

In [ ]:
import numpy as np

### Using pytorch

In [22]:
class PytorchNetwork(nn.Module):
    def __init__(self):
      super(PytorchNetwork, self).__init__()

      self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1)
      self.pool1 = nn.MaxPool2d(2, 2)
    
      self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=5, stride=1)
      self.pool2 = nn.MaxPool2d(2, 2)
    
      self.fc1 = nn.Linear(128, 128)
      self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
      x = self.conv1(x)
      x = F.relu(x)
      x = self.pool1(x)
    
      x = self.conv2(x)
      x = F.relu(x)
      x = self.pool2(x)
      
      x = torch.flatten(x, 1)

      x = self.fc1(x)
      x = F.relu(x)
      x = self.fc2(x)

      output = F.log_softmax(x, dim=1)
      return output

model = PytorchNetwork()
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

PytorchNetwork(
  (conv1): Conv2d(1, 4, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [34]:
for epoch in range(2):
    # trainning
    ave_loss = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        ave_loss = ave_loss * 0.9 + loss.item() * 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(
                epoch, batch_idx+1, ave_loss))
    # testing
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        ave_loss = ave_loss * 0.9 + loss.item() * 0.1
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_loader):
            print('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, ave_loss, correct_cnt * 1.0 / total_cnt))

==>>> epoch: 0, batch index: 100, train loss: 0.129008
==>>> epoch: 0, batch index: 200, train loss: 0.109498
==>>> epoch: 0, batch index: 300, train loss: 0.096006
==>>> epoch: 0, batch index: 400, train loss: 0.072754
==>>> epoch: 0, batch index: 500, train loss: 0.087649
==>>> epoch: 0, batch index: 600, train loss: 0.098350
==>>> epoch: 0, batch index: 100, test loss: 0.073826, acc: 0.975
==>>> epoch: 1, batch index: 100, train loss: 0.091941
==>>> epoch: 1, batch index: 200, train loss: 0.104930
==>>> epoch: 1, batch index: 300, train loss: 0.093259
==>>> epoch: 1, batch index: 400, train loss: 0.095452
==>>> epoch: 1, batch index: 500, train loss: 0.100227
==>>> epoch: 1, batch index: 600, train loss: 0.080403
==>>> epoch: 1, batch index: 100, test loss: 0.070209, acc: 0.977
==>>> epoch: 2, batch index: 100, train loss: 0.088849
==>>> epoch: 2, batch index: 200, train loss: 0.072697
==>>> epoch: 2, batch index: 300, train loss: 0.085524
==>>> epoch: 2, batch index: 400, train los

### Q2

In [ ]:
X_test = np.load("./Data/X_test.npy")
X_train = np.load("./Data/X_train.npy")
Y_test = np.load("./Data/Y_test.npy")
Y_train = np.load("./Data/Y_train.npy")